In [4]:
import torch
import torch.optim as optim

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

def model(t_u, w, b):
    return w * t_u + b # weight * x + bias => line

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples) # random

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

print(train_indices, val_indices)

# split data in train and validation sets
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

tensor([ 8,  3,  7, 10,  9,  4,  5,  0,  6]) tensor([1, 2])


In [22]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs+1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)

        # see also torch.set_grad_enabled(bool) that do the same when passing False
        with torch.no_grad(): # disable autograd graph building for these operations since we don't need to compute gradient
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad is False

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f'Epoch {epoch}, Training loss {train_loss.item():.4f}, Validation loss {val_loss.item():.4f}')
    return params

In [23]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(n_epochs=3000,
              optimizer=optimizer,
              params=params,
              train_t_u=train_t_un,
              val_t_u=val_t_un,
              train_t_c=train_t_c,
              val_t_c=val_t_c)

Epoch 1, Training loss 81.0008, Validation loss 77.5003
Epoch 2, Training loss 43.3337, Validation loss 20.9351
Epoch 3, Training loss 36.9531, Validation loss 8.1371
Epoch 500, Training loss 7.3156, Validation loss 2.3736
Epoch 1000, Training loss 3.7045, Validation loss 2.2572
Epoch 1500, Training loss 3.2517, Validation loss 2.2199
Epoch 2000, Training loss 3.1950, Validation loss 2.2072
Epoch 2500, Training loss 3.1878, Validation loss 2.2028
Epoch 3000, Training loss 3.1869, Validation loss 2.2012


tensor([  5.3077, -17.2599], requires_grad=True)